In [ ]:
import numpy
import torch
from FCNN import AE
from data_loader import NEFG3x3Set
import matplotlib.pyplot as plt
ml = AE()

loss_function = torch.nn.L1Loss()
dataset = NEFG3x3Set("info_dat_charge_std.csv",
                     "data_3x3_12", "dat_charge_std", transform=True)

In [ ]:

# 1600
# 1500 - example of very small data
# 1000 - very small nums
# 3000 - bigger numbers
(inp, tar, _,_,_)  = dataset[1500]
inp = inp.float()

rec, comp = ml(inp)
rec = rec.reshape((-1,26)).float()
fig, ax = plt.subplots(nrows=2, figsize=(8,12))
pos = ax[0].imshow(rec.detach().numpy())
c_bar = fig.colorbar(pos, ax=ax[0])
print(rec.min(), rec.max())

comp = comp.reshape((-1,10)).float()
pos = ax[1].imshow(comp.detach().numpy())
c_bar = fig.colorbar(pos, ax=ax[1])
# print(tar.min(), tar.max())

# inp = inp.reshape((-1,26)).float()
# pos = ax[2].imshow(inp.detach().numpy())
# c_bar = fig.colorbar(pos, ax=ax[2])
# print(inp.min(), inp.max())

# plt.show()
# print("Prediction Loss:", loss_function(tar, rec))

# print("Input Loss:", loss_function(tar, inp))



